<a href="https://colab.research.google.com/github/DIPANJAN001/Dipanjanpatra/blob/master/PV_BILSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ALL IMPORTS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install Boruta
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import concatenate
from keras import Model
from keras.layers import Input
from keras.layers import Bidirectional
from keras import layers
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_classification
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.layers import LeakyReLU
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.models import load_model
from keras.layers import Input
from sklearn.decomposition import PCA 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 1.8 MB/s 


### TIME SERIES TO SUPERVISED

In [2]:
def lstm_data_transform(x_data, y_data, num_steps):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = y_data[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

### DATA LOADING

In [ ]:
df=pd.read_excel("/content/pv_04.xlsx")
weather_input1=df.drop('power_normed',axis=1)
weather_input=weather_input1.drop('time_idx',axis=1)
solpow=df['power_normed']

### INPUTS

In [ ]:
num_steps = 3
EPOCH=100
VAL_SPLIT=0.2
BATCH_SIZE=64

### BORUTA FEATURE SELECTION

In [ ]:
rfc = RandomForestRegressor(random_state=1, n_estimators=1000, max_depth=5)
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2, random_state=1)
boruta_selector.fit(np.array(weather_input), np.array(solpow)) 

In [ ]:
X_important_train = boruta_selector.transform(np.array(weather_input))
# training set
(x_transformed_train,
 y_transformed_train) = lstm_data_transform(X_important_train,solpow , num_steps=num_steps)
assert x_transformed_train.shape[0] == y_transformed_train.shape[0]
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(x_transformed_train,y_transformed_train,test_size=0.4, random_state=42,shuffle=False)
#X_train_,X_val,y_train_,y_val=train_test_split(X_train,y_train,test_size=0.2, random_state=42,shuffle=False)

In [ ]:
inputs1 = Input(shape=(X_train1.shape[1],X_train1.shape[2]))

In [ ]:
def get_model11(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe1_0 = Bidirectional(LSTM(32, activation='relu',return_sequences = True))(inputs1)#32
    fe1_1 = Dropout(0.5)(fe1_0)
    fe1_2 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(fe1_1)#16
    fe1_3= Dropout(0.5)(fe1_2)
    fe1_4=Bidirectional(LSTM(8, activation='relu'))(fe1_3)#8
    out1_1=Dense(1, activation='linear')(fe1_4)
    return Model(inputs1, out1_1)
def get_model21(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe2_0 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(inputs1)#16,16,8
    fe2_1 = Dropout(0.5)(fe2_0)
    fe2_2 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(fe2_1)
    fe2_3= Dropout(0.5)(fe2_2)
    fe2_4=Bidirectional(LSTM(8, activation='relu'))(fe2_3)
    out2_1=Dense(1, activation='linear')(fe2_4)
    return Model(inputs1, out2_1)
def get_model31(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe3_0 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(inputs1)#16,8,8
    fe3_1 = Dropout(0.5)(fe3_0)
    fe3_2 = Bidirectional(LSTM(8, activation='relu',return_sequences = True))(fe3_1)
    fe3_3= Dropout(0.5)(fe3_2)
    fe3_4=Bidirectional(LSTM(8, activation='relu'))(fe3_3)
    out3_1=Dense(1, activation='linear')(fe3_4)
    return Model(inputs1, out3_1)
model1 = get_model11() 
model2 = get_model21() 
model3 = get_model31()
y1 = model1(inputs1) 
y2 = model2(inputs1) 
y3 = model3(inputs1)
outputs = layers.average([y1, y2, y3]) 
ensemble_model1= Model(inputs=inputs1, outputs=outputs)
ensemble_model1.compile(optimizer='Adam',loss='mean_squared_error',metrics=['RootMeanSquaredError'])
history=ensemble_model1.fit(X_train1, y_train1, epochs = EPOCH,validation_split=VAL_SPLIT,batch_size = BATCH_SIZE)

### TRAINING AND VALIDATIUON ERROR PLOT

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

### PREDICTION

In [ ]:
y1=ensemble_model1.predict(X_test1)
plt.scatter(y1,y_test1)
plt.show()

In [ ]:
df_ = pd.DataFrame()
df_['time']=[i for i in range(len(y1))]
df_['Actual']=y_test1
df_['Predicted']=y1
plt.plot(df_['time'],df_['Actual'])

plt.plot(df_['time'],df_['Predicted'])
plt.show()

### AUTOENCODER+BILSTM

In [ ]:
n_inputs=weather_input.shape[1]

In [ ]:
input_data_shape= Input(shape=(n_inputs,))
# encoder level 1
encoder= Dense(n_inputs*2)(input_data_shape)
encoder = BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
# encoder level 2
encoder= Dense(n_inputs)(encoder)
encoder= BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
# bottleneck
#n_bottleneck = round(float(n_inputs) / 2.0)
n_bottleneck = 20
bottleneck = Dense(n_bottleneck)(encoder)
# define decoder, level 1
decoder = Dense(n_inputs)(bottleneck)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)
# decoder level 2
decoder = Dense(n_inputs*2)(decoder)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)

In [ ]:
output = Dense(n_inputs, activation='linear')(decoder)
# define autoencoder model
model = Model(inputs=input_data_shape, outputs=output)
# compile autoencoder model
model.compile(optimizer='adam', loss='mse')
model.fit(weather_input,weather_input, epochs=100, batch_size=32, verbose=2)

In [ ]:
encoder = Model(inputs=input_data_shape, outputs=bottleneck)
# save the encoder to file
encoder.save('encoder.h5')
encoder = load_model('encoder.h5')

# encode the train data
X_train_encode = encoder.predict(weather_input)

In [ ]:
(x_transformed2,
 y_transformed2) = lstm_data_transform(X_train_encode,solpow , num_steps=num_steps)
assert x_transformed2.shape[0] == y_transformed2.shape[0]

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(x_transformed2, y_transformed2, test_size=0.4, random_state=42,shuffle=False)

In [ ]:
inputs2=Input(shape=(X_train2.shape[1],X_train2.shape[2]))

In [ ]:
def get_model12(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe1_0 = Bidirectional(LSTM(32, activation='relu',return_sequences = True))(inputs1)#32
    fe1_1 = Dropout(0.2)(fe1_0)
    fe1_2 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(fe1_1)#16
    fe1_3= Dropout(0.2)(fe1_2)
    fe1_4=Bidirectional(LSTM(8, activation='relu'))(fe1_3)#8
    out1_1=Dense(1, activation='linear')(fe1_4)
    return Model(inputs1, out1_1)
def get_model22(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe2_0 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(inputs1)#16,16,8
    fe2_1 = Dropout(0.5)(fe2_0)
    fe2_2 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(fe2_1)
    fe2_3= Dropout(0.5)(fe2_2)
    fe2_4=Bidirectional(LSTM(8, activation='relu'))(fe2_3)
    out2_1=Dense(1, activation='linear')(fe2_4)
    return Model(inputs1, out2_1)
def get_model32(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe3_0 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(inputs1)#16,8,8
    fe3_1 = Dropout(0.5)(fe3_0)
    fe3_2 = Bidirectional(LSTM(8, activation='relu',return_sequences = True))(fe3_1)
    fe3_3= Dropout(0.5)(fe3_2)
    fe3_4=Bidirectional(LSTM(8, activation='relu'))(fe3_3)
    out3_1=Dense(1, activation='linear')(fe3_4)
    return Model(inputs1, out3_1)
model1 = get_model12() 
model2 = get_model22() 
model3 = get_model32()
y1 = model1(inputs2) 
y2 = model2(inputs2) 
y3 = model3(inputs2)
outputs = layers.average([y1, y2, y3]) 
ensemble_model2 = Model(inputs=inputs1, outputs=outputs)
ensemble_model2.compile(optimizer='Adam',loss='mean_squared_error',metrics=['RootMeanSquaredError'])
history=ensemble_model2.fit(X_train2, y_train2, epochs = EPOCH,validation_split=VAL_SPLIT,batch_size = BATCH_SIZE)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='')
plt.legend()
plt.show()

In [ ]:
y2=ensemble_model2.predict(X_test1)
plt.scatter(y2,y_test2)
plt.show()

In [ ]:
df_ = pd.DataFrame()
df_['time']=[i for i in range(len(y2))]
df_['Actual']=y_test2
df_['Predicted']=y2
plt.plot(df_['time'],df_['Actual'])

plt.plot(df_['time'],df_['Predicted'])
plt.show()

### PCA

In [ ]:
pca = PCA(n_components = 16)
pca.fit(weather_input)
data_pca = pca.transform(weather_input)
data_pca = pd.DataFrame(data_pca)

In [ ]:
(x_transformed3,
 y_transformed3) = lstm_data_transform(data_pca,solpow , num_steps=num_steps)
assert x_transformed3.shape[0] == y_transformed3.shape[0]

In [ ]:
X_train3, X_test3, y_train3, y_test3= train_test_split(x_transformed3, y_transformed3, test_size=0.4, random_state=42,shuffle=False)

In [ ]:
inputs3=Input(shape=(X_train3.shape[1],X_train3.shape[2]))

In [ ]:
def get_model13(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe1_0 = LSTM(32, activation='relu',return_sequences = True)(inputs1)#32
    fe1_1 = Dropout(0.2)(fe1_0)
    fe1_2 = LSTM(16, activation='relu',return_sequences = True)(fe1_1)#16
    fe1_3= Dropout(0.2)(fe1_2)
    fe1_4=LSTM(8, activation='relu')(fe1_3)#8
    out1_1=Dense(1, activation='linear')(fe1_4)
    return Model(inputs1, out1_1)
def get_model23(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe2_0 = LSTM(16, activation='relu',return_sequences = True)(inputs1)#16,16,8
    fe2_1 = Dropout(0.5)(fe2_0)
    fe2_2 = LSTM(16, activation='relu',return_sequences = True)(fe2_1)
    fe2_3= Dropout(0.5)(fe2_2)
    fe2_4=LSTM(8, activation='relu')(fe2_3)
    out2_1=Dense(1, activation='linear')(fe2_4)
    return Model(inputs1, out2_1)
def get_model33(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe3_0 = LSTM(16, activation='relu',return_sequences = True)(inputs1)#16,8,8
    fe3_1 = Dropout(0.5)(fe3_0)
    fe3_2 = LSTM(8, activation='relu',return_sequences = True)(fe3_1)
    fe3_3= Dropout(0.5)(fe3_2)
    fe3_4=LSTM(8, activation='relu')(fe3_3)
    out3_1=Dense(1, activation='linear')(fe3_4)
    return Model(inputs1, out3_1)
model1 = get_model13() 
model2 = get_model23() 
model3 = get_model33()
y1 = model1(inputs3) 
y2 = model2(inputs3) 
y3 = model3(inputs3)
outputs = layers.average([y1, y2, y3]) 
ensemble_model3 = Model(inputs=inputs1, outputs=outputs)
ensemble_model3.compile(optimizer='Adam',loss='mean_squared_error',metrics=['RootMeanSquaredError'])
history=ensemble_model3.fit(X_train3, y_train3, epochs = EPOCH,validation_split=VAL_SPLIT,batch_size = BATCH_SIZE)

In [ ]:
y3=ensemble_model3.predict(X_test2)
plt.scatter(y3,y_test3)
plt.show()

In [ ]:
df_ = pd.DataFrame()
df_['time']=[i for i in range(len(y3))]
df_['Actual']=y_test3

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df_['time'],df_['Actual'])
plt.plot(df_['time'],df_['Predicted'])
plt.show()